In [ ]:
!pip install transformers

     |████████████████████████████████| 2.5MB 6.5MB/s 
     |████████████████████████████████| 3.3MB 29.9MB/s 
     |████████████████████████████████| 901kB 39.7MB/s 


In [ ]:
!pip install datasets

     |████████████████████████████████| 245kB 6.5MB/s 
     |████████████████████████████████| 122kB 8.2MB/s 
     |████████████████████████████████| 245kB 9.0MB/s 


In [ ]:
import numpy as np
import pandas as pd 
import tensorflow as tf 
from transformers import AutoTokenizer,TFAutoModelForSequenceClassification
from datasets import load_dataset

In [ ]:
raw_datasets=load_dataset('glue','mrpc')
checkpoint='bert-base-uncased'
tokenizer=AutoTokenizer.from_pretrained(checkpoint)
model=TFAutoModelForSequenceClassification.from_pretrained(checkpoint,num_labels=2)

Reusing dataset glue (/root/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
raw_datasets.keys()

dict_keys(['train', 'validation', 'test'])

In [ ]:
raw_datasets['train']

Dataset({
    features: ['idx', 'label', 'sentence1', 'sentence2'],
    num_rows: 3668
})

In [ ]:
raw_datasets['train'].features

{'idx': Value(dtype='int32', id=None),
 'label': ClassLabel(num_classes=2, names=['not_equivalent', 'equivalent'], names_file=None, id=None),
 'sentence1': Value(dtype='string', id=None),
 'sentence2': Value(dtype='string', id=None)}

In [ ]:
def tokenize_dataset(dataset):
    sent1=dataset['sentence1']
    sent2=dataset['sentence2']
    encoded=tokenizer(sent1,sent2,padding=True,truncation=True,return_tensors='np')
    return encoded.data
tokenized_dataset={split:tokenize_dataset(raw_datasets[split]) for split in raw_datasets.keys()}

In [ ]:
tokenized_dataset['train']

{'attention_mask': array([[1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        ...,
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0]]),
 'input_ids': array([[  101,  2572,  3217, ...,     0,     0,     0],
        [  101,  9805,  3540, ...,     0,     0,     0],
        [  101,  2027,  2018, ...,     0,     0,     0],
        ...,
        [  101,  1000,  2057, ...,     0,     0,     0],
        [  101,  1996, 26828, ...,     0,     0,     0],
        [  101,  1996,  2382, ...,     0,     0,     0]]),
 'token_type_ids': array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])}

In [ ]:
loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [ ]:
model.compile('adam',loss=loss,metrics=['accuracy'])

In [16]:
model.fit(tokenized_dataset['train'],np.array(raw_datasets['train']['label']),\
          validation_data=(tokenized_dataset['validation'],np.array(raw_datasets['validation']['label'])),batch_size=8)

459/459 [==============================] - 4319s 9s/step - loss: 0.6580 - accuracy: 0.6633 - val_loss: 0.6359 - val_accuracy: 0.6838


In [19]:
preds=model.predict(tokenized_dataset['validation']).logits

In [22]:
class_preds=tf.argmax(preds,axis=1)
print(preds.shape,class_preds.shape)

(408, 2) (408,)


In [23]:
from datasets import load_metric

In [24]:
metrics=load_metric('glue','mrpc')

In [25]:
metrics.compute(predictions=class_preds,references=raw_datasets['validation']['label'])

{'accuracy': 0.6838235294117647, 'f1': 0.8122270742358079}

In [26]:
class_preds[:5]

<tf.Tensor: shape=(5,), dtype=int64, numpy=array([1, 1, 1, 1, 1])>

In [27]:
raw_datasets['validation']['label'][:5]

[1, 0, 0, 1, 0]